# Sentiment Analysis with Naive-Bayes & Python

## Typical Supervised ML Workflow

1. Acquire Data
2. Preprocess / Clean Data
3. Build a feature set
4. Build a training model
5. Apply the model to tuning or test data
6. Score the results of the trained model
7. Revise hypothesis, train & test again until reach acceptable performance

## Background

Sentiment Analysis is a NLP technique for determining the
opinion polarity for a given text.

Let's apply this technique move reviews!

### What is the task?

For the following two reviews we'd expect to receive the two subsequent "labels"
 of their sentiment.

| Review  	                        | Label |
|---	                            |---	|
| "I love this movie!"   	        | `pos`	|
| "This movie really stinks :-("  	| `neg`	|

### First we need to ACQUIRE & PREPROCESS our data

A common dataset for training sentiment analysis algorithms
is the IMDB movie review dataset. It contains thousands of
movie reviews along with their sentiment polarity labeling (i.e., pos/neg.)

In [3]:
from nltk.corpus import movie_reviews

negative_ids = movie_reviews.fileids('neg')
positive_ids = movie_reviews.fileids('pos')

print(movie_reviews.sents(negative_ids[0]))

[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.'], ...]


Lets define a function to create our `features`. Features
are names given to data that can be used in a learning algorithm.
Features can be different types dependent on the algorithm being
used, but typically are binary or float values. Therefore, a
transform is necessary to convert our textual data into numerical
data.

In [ ]:
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
ps = PorterStemmer()


def word_feats(words):
    stop_words = set(stopwords.words('english'))
    return dict([(ps.stem(word), True) for word in words if not word in stop_words])

Now, create the positive and negative `features`

In [23]:
negative_features = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negative_ids]
positive_features = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in positive_ids]

This creates two lists of dictionaries, where every dict
corresponds to the set of words found in a particular positive or negative
document.

Next, we need to split our labeled data into training and
testing data sets. Why? We want to be able to test how accurate
the model we are going to develop is, in order to do that we
need labeled data to test on. An 80/20 split is typical.

In [24]:
neg_cutoff = round(len(negative_features) * 0.80)
print(neg_cutoff)
pos_cutoff = round(len(positive_features) * 0.80)
print(pos_cutoff)
training_features = negative_features[:neg_cutoff] + positive_features[:pos_cutoff]
testing_features = negative_features[neg_cutoff:] + positive_features[pos_cutoff:]
print('train on %d instances, test on %d instances' % (len(training_features), len(testing_features)))

800
800
train on 1600 instances, test on 400 instances


## Classification
We're ready to train our model. One of the simplest Machine Learning algorithms is the Naive Bayes Classifier.

In [25]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(training_features)
print('accuracy:', nltk.classify.util.accuracy(classifier, testing_features))

accuracy: 0.71


Can we get any sense of how these decisions are being made?

In [19]:
classifier.show_most_informative_features()

Most Informative Features
             outstanding = True              pos : neg    =     13.9 : 1.0
               insulting = True              neg : pos    =     13.7 : 1.0
              vulnerable = True              pos : neg    =     13.0 : 1.0
               ludicrous = True              neg : pos    =     12.6 : 1.0
             uninvolving = True              neg : pos    =     12.3 : 1.0
              astounding = True              pos : neg    =     11.7 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             fascination = True              pos : neg    =     11.0 : 1.0
               affecting = True              pos : neg    =     10.3 : 1.0
               animators = True              pos : neg    =     10.3 : 1.0


Okay, cool. What about on some new data?

In [26]:
test_reviews = [
"""Wow! That's about all one can say about this movie. The first time that I saw
it I was mesmerized. The movie looked so cool and hey, it actually had a good
plot. If you haven't seen this movie yet, get out from your cave and see it
right away. I have seen this movie umpteen times and it still shocks and
surprises me.""",
"""Anyway, back to the movie. It is as bad as you've no doubt heard. The scene
changes from night to day to night, the spaceship is a hubcap (you can see the
string it hangs from catch on fire at one point), I could do a better job
acting, etc. """]

from nltk.tokenize import word_tokenize

for review in test_reviews:
    review_features = word_feats(word_tokenize(review.lower()))
    label = classifier.classify(review_features)
    prob_results = classifier.prob_classify(review_features)
    prob_str = " ({0:.2}/{1:.2})".format(prob_results.prob("pos"), prob_results.prob("neg"))
    print(review[:25], ": ", label, prob_str)


Wow! That's about all one :  neg  (0.31/0.69)
Anyway, back to the movie :  neg  (0.38/0.62)


Decision Tree, Support Vector Machines. Ways of improving the features, tfidf